In [ ]:
import bisect
import datetime as dt
import functools
import glob
import keras
import math
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import more_itertools as mit
import networkx as nx
import numpy as np
import operator
import os
import pandas as pd
import pickle
import random
import re
import scipy.stats
import seaborn as sns
import shutil
import statsmodels as sm
import time
import torch
import torch.utils.data
import warnings
import xgboost as xgb
from datetime import date, timedelta, datetime
from itertools import chain, permutations
from keras import optimizers
from keras.layers import Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from matplotlib import pyplot as plt
from numpy import loadtxt, matrix
from random import randint
from scipy import stats
from scipy.stats import ks_2samp
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.extmath import cartesian
from statsmodels.distributions.empirical_distribution import ECDF
from tensorflow.keras.layers import Activation
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from time import time, strftime, gmtime
from torch_geometric.data import Data, DataLoader, InMemoryDataset
from torch_geometric.datasets import Reddit
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 5000)


In [ ]:
lag=0

In [ ]:
dc = pd.read_csv("C:\\Users\\szahedi1\\Amir_Thesis\\data_09_17_21.csv", delimiter=',' ,dtype=None)

In [ ]:
# df.drop_duplicates(subset=['measurement_tstamp',"pair"],inplace=True,keep='first',ignore_index=True)
#df=dc.copy()
df = dc.copy()
df.drop(['roadname_up','xd_up','xd_dn','tmc_up','tmc_dn','roadname_dn','Intersection Name'],axis=1,inplace=True)
df=df.sort_values(by='speed_up', ascending=True).reset_index(drop=True).reset_index()
# df = df.loc[df['l_veh'].notna()]
# dg = df[['measurement_tstamp','Detector_ID','dir','pair']].groupby(['measurement_tstamp','Detector_ID',
#                                                                     'dir']).count().reset_index()
# dg = dg.rename(columns={"pair":"approach_obs"})
# df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left').set_index('index')
# df['index']=df.index
# df = df[df["movements"]==df["approach_obs"]]
# df['h_veh'] = df['h_veh'].fillna(0)
# df['count'] = df['h_veh'] + df['l_veh']
# df = df.drop(['l_veh','h_veh'],axis=1)
# dg = df.groupby(['measurement_tstamp','Detector_ID','dir']).sum()
# dg=dg.reset_index()
# dg = dg[['measurement_tstamp','Detector_ID','dir','count']]
# dg = dg.rename(columns={"count":"approach_count"})
# df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left').set_index('index').drop(['approach_obs'],axis=1)
df['index']=df.index
df

In [ ]:
df[df['ref_speed_dn'].isna()]

In [ ]:
df = df.drop(['Unnamed: 0'],axis=1)

In [ ]:
df['time']=(df['hour']+df['quarter']/4)

dum_df = pd.get_dummies(df, columns=['dir','Dn','year','month','weekday','time'] )
#
dum_df = dum_df.drop(['pair','measurement_tstamp','hour', 'quarter','day',
                      'Up','Detector_ID','Direction','Exclusive_Lane',
                      'lanes_up','lanes_dn','ref_speed_up','speed_dn','avg_speed_up','avg_speed_dn',
                      
                     'movements','move','index','l_veh','h_veh','speed_up'],axis=1)

dum_df

In [ ]:
Epochs=5

train=dum_df.loc[dum_df['ref_speed_dn'].notna()]
print(train.shape)
test =dum_df
print(test.shape)


X_train=train.drop(['ref_speed_dn'],axis=1)
print(X_train.shape)
X_test=test.drop(['ref_speed_dn'],axis=1)
print(X_test.shape)
y_train=train['ref_speed_dn']
print(y_train.shape)
y_test=test['ref_speed_dn']
print(y_test.shape)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train= scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()

model.add(Dense(128, input_dim=224, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
#model.add(Dropout(0.10))

model.add(Dense(128, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
#model.add(Dropout(0.10))

model.add(Dense(128, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
#model.add(Dropout(0.10))

model.add(Dense(128, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
#model.add(Dropout(0.10))

model.add(Dense(1, activation='linear'))

model.summary()
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae','mape'])
history = model.fit(X_train, y_train, epochs=Epochs, batch_size=8096,  verbose=1, validation_data = (X_test, y_test))


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
df['pred'] = model.predict(X_test)

In [ ]:
df.head()

In [ ]:
dc = dc.merge(df[['pair','measurement_tstamp','Dn','pred']],on=['pair','measurement_tstamp','Dn'],how='left')
dc

In [ ]:
dc.ref_speed_dn.fillna(dc.pred, inplace=True)
dc= dc.drop(['pred'],axis=1)
dc

In [ ]:
dc[dc['ref_speed_dn'].isna()]

In [ ]:
dc = dc.drop(['Unnamed: 0'],axis=1)
dc

In [ ]:
dc.to_csv('C:\\Users\\szahedi1\\Amir_Thesis\\data_09_17_21.csv', sep=',')

In [ ]:
train=dum_df.loc[dum_df['speed_up'].notna()]
test =dum_df
print(train.shape,test.shape)


X_train=train.drop(['speed_up'],axis=1)
X_test=test.drop(['speed_up'],axis=1)

y_train=train['speed_up']
y_test=test['speed_up']

regressor = xgb.XGBRegressor(n_estimators=50,reg_lambda=1, gamma=0,max_depth=9, learning_rate=0.1)
regressor.fit(X_train, y_train)
#res=test.merge(pd.DataFrame(data={'actual':y_test,'pred':regressor.predict(X_test)}), how='left', left_index=True, right_index=True)


In [ ]:
res=test.merge(pd.DataFrame(data={'actual':y_test,'pred':regressor.predict(X_test)}), how='left', left_index=True, right_index=True)
res

In [ ]:
df = df.merge(res[['pred']], left_index=True, right_index=True,how='left')
df

In [ ]:
df['MAPE'] = abs(df['pred']-df['speed_up'])/df['speed_up']
df.MAPE.mean()

In [ ]:
df.dif.sum()

In [ ]:
res = pd.wide_to_long(res.reset_index(),  stubnames='Up_', i='index', j='Up')
res = res.loc[res['Up_']==1]
res = pd.wide_to_long(res.reset_index(),  stubnames='weekday_', i='index', j='weekday')
res = res.loc[res['weekday_']==1]
res = pd.wide_to_long(res.reset_index(),  stubnames='month_', i='index', j='month')
res = res.loc[res['month_']==1]
res = pd.wide_to_long(res.reset_index(),  stubnames='time_', i='index', j='time')
res = res.loc[res['time_']==1]
res = pd.wide_to_long(res.reset_index(),  stubnames='year_', i='index', j='year')
res = res.loc[res['year_']==1]
res

In [ ]:
res = res.drop(['weekday_','month_','Up_','time_','year_'],axis=1)


In [ ]:
res['dir']=res['dir_NB']*4+res['dir_WB']*3+res['dir_EB']*2+res['dir_SB']
res['dir'] = ['NB' if x ==4 else 'WB' if x ==3 else 'EB' if x ==2 else 'SB'for x in res["dir"]]
res = res.drop(['dir_NB','dir_WB','dir_EB','dir_SB'],axis=1)
res = res.reset_index().drop(['index'],axis=1)
res

In [ ]:
res = res.reset_index().drop(['index','actual','speed_up'],axis=1)
res

In [ ]:
res = res.reset_index().drop(['actual','speed_up'],axis=1)
res

In [ ]:
df=df.merge(res,on=['year','month','weekday','time','Up','dir'],how='left')
df

In [ ]:
pd.wide_to_long(ds.reset_index(),  stubnames='Course_0', i='index', j='Course')


In [ ]:
ndf = pd.wide_to_long(ds.reset_index(), stubnames='Course_',i='index',j='T')
#not_dummy = ndf[ndf['T_'].ne(0)].reset_index(level='T').drop('T_',1)
ndf

In [ ]:
if (i==1):
    results=res
else:
    results=pd.concat([results,res])
if (len(results.loc[results['actual']-results['nextcount']!=0])==0):
    print('Done')
else:
    print('there is a problem')
# count
result = pd.DataFrame(columns = ["Detector_ID","XGBoost_R_Squared","XGBoost_MAPE","XGBoost_MAE"])
result=result.append({"Detector_ID":str(loc),
                          "XGBoost_R_Squared":r2_score(results.loc[results['Detector_ID']==loc,'nextcount'], results.loc[results['Detector_ID']==loc,'pred'] ),
                          "XGBoost_MAPE": (abs(results.loc[results['Detector_ID']==loc,'nextcount']-results.loc[results['Detector_ID']==loc,'pred'])/results.loc[results['Detector_ID']==loc,'nextcount']).mean()*100,
                          "XGBoost_MAE": (abs(results.loc[results['Detector_ID']==loc,'nextcount']-results.loc[results['Detector_ID']==loc,'pred'])).mean()},
                       ignore_index=True)

result

In [ ]:
df = df.loc[df['l_veh'].notna()]
dg = df[['measurement_tstamp','Detector_ID','dir','pair']].groupby(['measurement_tstamp','Detector_ID',
                                                                    'dir']).count().reset_index()
dg = dg.rename(columns={"pair":"approach_obs"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left')
df = df[df["movements"]==df["approach_obs"]]
df['h_veh'] = df['h_veh'].fillna(0)
df['count'] = df['h_veh'] + df['l_veh']
df = df.drop(['l_veh','h_veh'],axis=1)
dg = df.groupby(['measurement_tstamp','Detector_ID','dir']).sum().reset_index()
dg = dg[['measurement_tstamp','Detector_ID','dir','count']]
dg = dg.rename(columns={"count":"approach_count"})
df = df.merge(dg, on=["Detector_ID","measurement_tstamp","dir"], how='left').drop(['approach_obs'],axis=1)
# df = df[df['speed_up'].notna()]
# df = df[df['speed_dn'].notna()]


In [ ]:
df[(df['approach_count'].notna())&(df['speed_up'].isna())]

In [ ]:
dg = dc.copy()
dg.drop(['Unnamed: 0','Unnamed: 0.1','roadname_up','xd_up','xd_dn','tmc_up','tmc_dn','roadname_dn','Intersection Name',
        'l_veh','h_veh'],axis=1,inplace=True)
dg.drop_duplicates(subset=['measurement_tstamp',"pair"],inplace=True,keep='first',ignore_index=True)
dg['date'] = pd.to_datetime(dg['measurement_tstamp']).dt.date
df = dg.merge(df[['pair','measurement_tstamp','count','approach_count']],on=['pair','measurement_tstamp'],how='left')
df['measurement_tstamp'] = pd.to_datetime(df.measurement_tstamp)
df['week'] = df.measurement_tstamp.dt.week
df.shape

In [ ]:
dg = pd.DataFrame()
dg = df[['pair','measurement_tstamp','count']]
dg = dg.rename(columns={'count':'nextcount'})
dg['measurement_tstamp'] = pd.to_datetime(dg['measurement_tstamp'])
dg['measurement_tstamp'] -= timedelta(hours=0, minutes=lag)
df['measurement_tstamp'] = pd.to_datetime(df['measurement_tstamp'])
df = df.merge(dg, on=['pair','measurement_tstamp'],how='left')

## Addition of downstream volume

In [ ]:
df['dn_dir'] = ['NB' if x =="NB_TH" else 'NB' if x =="WB_RT" else 'NB' if x =="EB_LT" else
                'SB' if x =="SB_TH" else 'SB' if x =="EB_RT" else 'SB' if x =="WB_LT" else
                'EB' if x =="EB_TH" else 'EB' if x =="NB_RT" else 'EB' if x =="SB_LT" else 'WB'for x in df["Direction"]]
dh = df[df['count'].notna()]
dh = dh.groupby(['Detector_ID','measurement_tstamp','dn_dir']).count().reset_index()
dh = dh[['Detector_ID','measurement_tstamp','dn_dir','pair']]
dh = dh.rename(columns={'pair':'dn_moves'})
dg = df.groupby(['Detector_ID','measurement_tstamp','dn_dir']).count().reset_index()
dg = dg[['Detector_ID','measurement_tstamp','dn_dir','pair']]
dg = dg.rename(columns={'pair':'dn_max_moves'})

dg = dg.merge(dh,on=['Detector_ID','measurement_tstamp','dn_dir'],how='left')
dg = dg[dg['dn_max_moves']==dg['dn_moves']]


dg=df.merge(dg, how='inner', on=['Detector_ID','measurement_tstamp','dn_dir'])
dg = dg[['Detector_ID','measurement_tstamp','dn_dir','count']].groupby(['Detector_ID','measurement_tstamp',
                                                                        'dn_dir']).sum().reset_index()
dg = dg.rename(columns={'count':'dn_count'})
df = df.merge(dg, how='inner', on=['Detector_ID','measurement_tstamp','dn_dir'])
df.head()

In [ ]:
dg = df.dropna()
dg['weekend'] = [0 if x <5 else 1 for x in dg["weekday"]]
dg.head()

In [ ]:
dq = dg.dropna().groupby(['hour','Detector_ID','weekend','pair']).Dn.count().reset_index()

In [ ]:
dq['days'] = [5 if x ==0 else 2 for x in dq["weekend"]]

In [ ]:
dq['Dn1']=dq['Dn']/dq['days']/4

In [ ]:
dq.sort_values(by=['Dn1']).tail(4000)

In [ ]:
dq.loc[dq['Dn1']>30].groupby(['hour']).count()

In [ ]:
len(dq.loc[(dq['Dn1']>14)&(dq['weekend']==1)].pair.unique())

In [ ]:
listdet = list(dq.loc[dq['Dn1']>14].Detector_ID.unique())
listdet

In [ ]:
len(dg.pair.unique())

In [ ]:
dg = dg.loc[dg['Detector_ID'].isin(listdet)]

In [ ]:
dg = dg.loc[(dg['hour']>6)&(dg['hour']<20)]
dq = dg.groupby(['Detector_ID','Direction','hour']).pair.count().reset_index().sort_values(by='pair').reset_index()
dq= dq[dq['pair']<56]
dq['comp'] = dq['Detector_ID'].astype(int).astype(str) + dq['Direction'].astype(str) + dq['hour'].astype(str)
dq.head()

In [ ]:
dg['comp'] = dg['Detector_ID'].astype(int).astype(str) + dg['Direction'].astype(str) + dg['hour'].astype(str)
dg = dg.loc[~dg['comp'].isin(list(dq.comp.unique()))]
dg.head()

In [ ]:
dq = dg.groupby(['Detector_ID','Direction','hour','quarter','weekday']).pair.count().reset_index().sort_values(by='pair').reset_index()
dg['comp'] = dg['Detector_ID'].astype(int).astype(str) + dg['Direction'].astype(str) + dg['weekday'].astype(str)+ dg['hour'].astype(str) + dg['quarter'].astype(str)
dg = dg.sort_values(by=['pair','measurement_tstamp'])
dq = dg.drop_duplicates(subset=['comp'],keep='first',ignore_index=True)
dq = dq[['pair','measurement_tstamp']]
dq['ind']=1
dg = dg.merge(dq,on=['pair','measurement_tstamp'],how='left')
dg

In [ ]:

dum_df = pd.get_dummies(dg, columns=["hour",'quarter','weekday','dir','move','dn_dir'] )

dum_df = dum_df.drop(['pair','measurement_tstamp','year','month','day',
                      'Up','Dn','Direction',
                      'lanes_up','lanes_dn',
                     'movements','date','count','week','comp'],axis=1)

dum_df

In [ ]:
result = pd.DataFrame(columns = ["Detector_ID","Training_Set_Size","Validation_Set_Size",
                                 "Train_MSE","Validation_MSE",
                                 "Train_MAE","Validation_MAE",
                                 "Train_MAPE","Validation_MAPE",
                                 "Train_R_Squared","Validation_R_Squared"])

Epochs=50
for loc in dum_df.Detector_ID.unique().tolist():
    train=dum_df.loc[(dum_df['Detector_ID']==loc)&(dum_df['ind']==1)]
    test =dum_df.loc[(dum_df['Detector_ID']==loc)&(dum_df['ind']!=1)]
    print(loc,train.shape,test.shape)

    
    X_train=train.drop(['Detector_ID','nextcount','ind'],axis=1)
    X_test=test.drop(['Detector_ID','nextcount','ind'],axis=1)
    
    y_train=train['nextcount']
    y_test=test['nextcount']
    
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train= scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = Sequential()
    
    model.add(Dense(256, input_dim=46, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    #model.add(Dropout(0.10))
    
    model.add(Dense(256, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    #model.add(Dropout(0.10))
    
    model.add(Dense(256, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    #model.add(Dropout(0.10))
    
    model.add(Dense(256, use_bias=False))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    #model.add(Dropout(0.10))
    
    model.add(Dense(1, activation='linear'))
    
    model.summary()
    model.compile(loss='mae', optimizer='adam', metrics=['mse','mae','mape'])
    history = model.fit(X_train, y_train, epochs=Epochs, batch_size=32,  verbose=1, validation_data = (X_test, y_test))
    r2_train = 1 - (np.sum((np.asarray(y_train).reshape(y_train.shape[0],1) - model.predict(X_train))**2))/(np.sum((np.asarray(y_train).reshape(y_train.shape[0],1) - np.mean(y_train))**2))
    r2_test = 1 - (np.sum((np.asarray(y_test).reshape(y_test.shape[0],1) - model.predict(X_test))**2))/(np.sum((np.asarray(y_test).reshape(y_test.shape[0],1) - np.mean(y_test))**2))
    
    
    result = result.append({"Detector_ID":loc,
                            "Training_Set_Size":X_train.shape[0],
                            "Validation_Set_Size":X_test.shape[0],
                            "Train_MSE": history.history['mse'][Epochs-1],
                            "Validation_MSE": history.history['val_mse'][Epochs-1],
                            "Train_MAE": history.history['mae'][Epochs-1],
                            "Validation_MAE": history.history['val_mae'][Epochs-1],
                            "Train_MAPE": history.history['mape'][Epochs-1],
                            "Validation_MAPE": history.history['val_mape'][Epochs-1],
                            "Train_R_Squared": r2_train,
                            "Validation_R_Squared":r2_test},
                           ignore_index=True)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

result

# Naive model prediction

In [ ]:
dq = dg.loc[dg['ind']==1].copy()
dq['nextproportion'] = dq['count']/dq['approach_count']
dq['measurement_tstamp'] -= timedelta(hours=0, minutes=lag)
dum_df = dg.loc[dg['ind']!=1].copy()
dum_df = dum_df.merge(dq[['pair','hour','quarter','weekday','nextproportion']],on=['pair','hour','quarter','weekday'])
dum_df

In [ ]:
dum_df['count_naive'] = dum_df['approach_count']*dum_df['nextproportion']
dum_df['count_diff_h0'] = (dum_df['count_naive']-dum_df['nextcount'])
dum_df['Naive_MAE'] = abs(dum_df['count_diff_h0'])
dum_df['Naive_MAPE'] = abs(dum_df['count_diff_h0'])/dum_df['nextcount']*100


db = dum_df[['Detector_ID','Naive_MAPE','Naive_MAE']].groupby(['Detector_ID']).mean().reset_index()
i=0
result['Naive_R_Squared']=0
for loc in dum_df.Detector_ID.unique().tolist():
    result.iloc[i,4] = {scipy.stats.linregress(x=dum_df[dum_df['Detector_ID']==loc]['count_naive'], y=dum_df[dum_df['Detector_ID']==loc]['nextcount']).rvalue ** 2}
    i+=1
result['Detector_ID'] = pd.to_numeric(result['Detector_ID'])
result= result.merge(db, on=['Detector_ID'])
result

In [ ]:
dg = pd.read_csv("C:\\Users\\szahedi1\\Amir_Thesis\\results(halfsmoothall).csv", delimiter=',' ,dtype=None)
dg = dg.loc[dg['Horizon']==0]
dg.drop(['Horizon'],axis=1, inplace=True)
results = result[["Detector_ID","Training_Set_Size","Validation_Set_Size","Validation_R_Squared","Validation_MAPE","Validation_MAE"]].merge(dg,on=['Detector_ID'],how='left')
results.mean()

In [ ]:
diff = pd.DataFrame()
diff['Detector_ID'] = results['Detector_ID']
diff['R2_X_N']    =   results['XGBoost_R_Squared']    - results['Naive_R_Squared']
diff['R2_NN_N']   =   results['Validation_R_Squared'] - results['Naive_R_Squared']
diff['R2_NN_X']   =   results['Validation_R_Squared'] - results['XGBoost_R_Squared']
diff['MAPE_X_N']  = -(results['XGBoost_MAPE']         - results['Naive_MAPE'])
diff['MAPE_N_N']  = -(results['Validation_MAPE']      - results['Naive_MAPE'])
diff['MAPE_NN_X'] = -(results['Validation_MAPE']      - results['XGBoost_MAPE'])
diff['MAE_X_N']   = -(results['XGBoost_MAE']          - results['Naive_MAE'])
diff['MAE_NN_N']  = -(results['Validation_MAE']       - results['Naive_MAE'])
diff['MAE_NN_X']  = -(results['Validation_MAE']       - results['XGBoost_MAE'])
diff

In [ ]:
diff.mean()